In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report


In [2]:
df = pd.read_csv('/content/drive/MyDrive/IS Project/ML/hypertension_dataset.csv')
df.head()

,Age,Salt_Intake,Stress_Score,BP_History,Sleep_Duration,BMI,Medication,Family_History,Exercise_Level,Smoking_Status,Has_Hypertension
0,69,8.0,9,Normal,6.4,25.8,NaN,Yes,Low,Non-Smoker,Yes
1,32,11.7,10,Normal,5.4,23.4,NaN,No,Low,Non-Smoker,No
2,78,9.5,3,Normal,7.1,18.7,NaN,No,Moderate,Non-Smoker,No
3,38,10.0,10,Hypertension,4.2,22.1,ACE Inhibitor,No,Low,Non-Smoker,Yes
4,41,9.8,1,Prehypertension,5.8,16.2,Other,No,Moderate,Non-Smoker,No


In [4]:
print("ค่าว่าง")
df.isnull().sum()

ค่าว่าง


,0
Age,0
Salt_Intake,0
Stress_Score,0
BP_History,0
Sleep_Duration,0
BMI,0
Medication,799
Family_History,0
Exercise_Level,0
Smoking_Status,0


In [5]:
df.dtypes

,0
Age,int64
Salt_Intake,float64
Stress_Score,int64
BP_History,object
Sleep_Duration,float64
BMI,float64
Medication,object
Family_History,object
Exercise_Level,object
Smoking_Status,object


In [6]:
df['Medication'] = df['Medication'].fillna('None')
df = df.replace({'Yes': 1, 'No': 0, 'Smoker': 1, 'Non-Smoker': 0})
df['Exercise_Level'] = df['Exercise_Level'].map({'Low': 0, 'Moderate': 1, 'High': 2})
df['BP_History'] = df['BP_History'].map({'Normal': 0, 'Prehypertension': 1, 'Hypertension': 2})

/tmp/ipython-input-762/304190670.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Yes': 1, 'No': 0, 'Smoker': 1, 'Non-Smoker': 0})


In [7]:
df.isnull().sum()

,0
Age,0
Salt_Intake,0
Stress_Score,0
BP_History,0
Sleep_Duration,0
BMI,0
Medication,0
Family_History,0
Exercise_Level,0
Smoking_Status,0


In [8]:
df = pd.get_dummies(df, columns=['Medication'], drop_first=True)

In [9]:
X = df.drop(columns=['Has_Hypertension'])
y = df['Has_Hypertension']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
model_lr = LogisticRegression()
model_knn = KNeighborsClassifier(n_neighbors=5)
model_dt = DecisionTreeClassifier(max_depth=5, random_state=42)

In [11]:
ensemble_model = VotingClassifier(
    estimators=[('lr', model_lr), ('knn', model_knn), ('dt', model_dt)],
    voting='soft'
)
ensemble_model.fit(X_train_scaled, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('dt',
                              DecisionTreeClassifier(max_depth=5,
                                                     random_state=42))],
                 voting='soft')

In [14]:
y_pred = ensemble_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred) * 100
print(f"\nAccuracy Ensemble (3 model): {accuracy:.2f}%")
print("\nรายละเอียดการทายผล ")
print(classification_report(y_test, y_pred))


Accuracy Ensemble (3 model): 88.92%

รายละเอียดการทายผล 
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       192
           1       0.94      0.84      0.89       205

    accuracy                           0.89       397
   macro avg       0.89      0.89      0.89       397
weighted avg       0.89      0.89      0.89       397



In [15]:
import joblib
joblib.dump(ensemble_model, 'ensemble_3_models.pkl')
joblib.dump(scaler, 'scaler_3_models.pkl')
print("\n✅ บันทึกไฟล์ .pkl และรูปกราฟสำเร็จ!")


✅ บันทึกไฟล์ .pkl และรูปกราฟสำเร็จ!
